### Bilanzen

In diesem Programm sind die Energie- Massen- und jegliche anderen Bilanzen wie Kalkulationen ausgeführt. Diese werden im Main-Programm aufs neue aufgerufen.

### Energiebilanzen

Erstmal ohne Auflösung der Temperatur, da die Darstellung nichtlinear wäre und das die Komplexität des Modells strak beeinflusst.

In [ ]:
### Quelle

# Energiebilanz Wärmestrom aus der Quelle
#model.Con_Quelle_Aus = Constraint( expr = model.Quelle_H_Aus[i,l,z] == model.Quelle_m_Aus[i,l,z] * model.cp_w * (model.T_Quelle[i,l,z] +273.15))
model.Con_Quelle_Aus = Constraint( expr = model.Quelle_H_Aus[i,l,z] <= model.Quelle_H_max)
            

### Wärmepumpe

# Bilanzen in die Wärmepumpe hinein
model.Con_WP_Eingang_T = Constraint( expr = model.WP_T_Verdampfer[i,l,z] == model.T_Quelle[i,l,z])
model.Con_WP_Ein_H = Constraint ( expr = model.WP_Q_Verdampfer[i,l,z] <= model.Quelle_H_Aus[i,l,z])


# Energiebilanz in der Wärmepumpe
model.Con_WP_COP = Constraint( expr = model.WP_Q_Kondensator[i,l,z] == model.WP_COP * model.WP_P_elektrisch[i,l,z])
model.Con_WP_EB = Constraint( expr = model.WP_Q_Kondensator[i,l,z] == model.WP_P_elektrisch[i,l,z] + model.WP_Q_Verdampfer[i,l,z])
'Q_Verdampfer hat keine EB'
model.C_WP_T_balance = Constraint( expr = (model.WP_T_Kondensator[i,l,z] + 273.15) == (model.WP_T_Verdampfer[i,l,z] + 273.15) + model.WP_Q_Kondensator[i,l,z]*1000/model.WP_COP_carnot)

model.Con_WP_P_max = Constraint ( expr = model.WP_P_elektrisch[i,l,z] <= model.WP_P_max)
#model.C_WP_T_Kondensator_max = Constraint(expr = model.WP_T_Kondensator[i,l,z] <= model.WP_T_Kondensator_max)



# Energiebilanz aus der Wärmepumpe heraus
#model.C_WP_Ausgang = Constraint( expr = model.WP_Q_Kondensator[i,l,z] == model.WP_m_Kondensator[i,l,z] * model.cp_w * (model.WP_T_Kondensator[i,l,z] + 273.15))
'Nichtlineare Gleichung'

# Kalkulation Wärmepumpe
model.C_WP_capex = Constraint( expr = model.WP_capex[i,l,z] == model.WP_P_elektrisch[i,l,z] * model.WP_invest) # Variabel von der Lesitung            
model.C_WP_opex = Constraint( expr = model.WP_opex[i,l,z] == model.WP_P_elektrisch[i,l,z] * 8760 * model.c_electricity * 0.01)            
                       

# Energiebilanz Wärmepumpe-Leitung
#model.Con_Leitung_EB = Constraint ( expr = model.L_H_Ein[i,l,z] == model.WP_Q_Kondensator[i,l,z] + model.Quelle_H_Aus[i,l,z])
model.Con_Leitung_EB = Constraint ( expr = model.L_H_Ein[i,l,z] == model.WP_Q_Kondensator[i,l,z])

#model.Con_Leitung_H_Ein = Constraint( expr = model.L_H_Ein[i,l,z] == model.m_L_Ein[i,l,z] * model.cp_w * (model.L_T_Ein[i,l,z] + 273.15))
'Nichtlineare Gleichung'
model.Con_Leitung_T_Ein = Constraint( expr = model.WP_T_Kondensator[i,l,z] == model.L_T_Ein[i,l,z])

# Massenbilanz zur Leitung
#model.Con_Leitung_MB = Constraint( expr = model.m_L_Ein[i,l,z] == model.WP_m_Kondensator[i,l,z] + model.Quelle_m_Aus[i,l,z])

### Leitung

# Energiebilanz der Leitungsverluste
model.Con_Leitung_Verlust = Constraint( expr = model.L_H_Aus[i,l,z] == model.L_H_Ein[i,l,z] - model.L_Q_Verlust[i,l,z]*0.001)
#model.Con_Leitung_H_Aus = Constraint( expr = model.L_H_Aus[i,l,z] == model.m_L_Ein[i,l,z] * model.cp_w * (model.L_T_Aus[i,l,z] + 273.15))
'Nichtlineare Gleichung'
model.Con_Leitung_Q_Verlust = Constraint( expr = model.L_Q_Verlust[i,l,z] == model.L_U * model.L_l[i,l,z] * (0.5 * (model.L_T_Ein[i,l,z] + model.L_T_Aus[i,l,z]) - model.T_Boden))
model.Con_Leitung_T_Verlust = Constraint( expr = model.L_T_Aus[i,l,z] == model.L_T_Ein[i,l,z] * (1-0.15))
'Aktuelle Annahme: Leitungsverlust von 15% -> m*c bleibt konstant, daher verlieren wir 15% der Temperatur'
# Kalkulation Leitung            
model.Con_Leitung_Capex = Constraint( expr = model.L_capex[i,l,z] == model.k_Leitung * model.L_l[i,l,z])

### Siedlung

# Energiebilanz
model.Con_Siedlung_EB = Constraint( expr = model.S_Q_Ein[i,l,z] == model.L_H_Aus[i,l,z])
model.Con_Siedlung_T = Constraint( expr = model.S_T_Ein[i,l,z] == model.L_T_Aus[i,l,z])
#model.Con_Siedlung_H_Ein = Constraint( expr = model.S_Q_Ein[i,l,z] == model.m_L_Ein[i,l,z] * model.cp_w * (model.S_T_Ein[i,l,z] + 273.15))
'Nichtlineare Gleichung'                                      
model.C_S_T = Constraint( expr = model.S_T_Ein[i,l,z] == model.S_T_Soll)
           
#model.C_S_Q = Constraint( expr = model.S_Q_Ein[i,l,z] >= model.S_Q_min / 8760)
model.Con_Siedlung_H = Constraint( expr = model.S_Q_Ein[i,l,z] == model.S_n[i,l,z] * model.S_Q_Haus)
model.C_VB_W = Constraint( expr = model.S_W_gesamt[i,l,z] == model.S_Q_Ein[i,l,z] * 8760)

# Kalkulation Siedlung
model.C_A_capex = Constraint( expr = model.A_capex[i,l,z] == model.A_k * model.S_n[i,l,z])